In [1]:
import pandas as pd

final_dataset_loc = 'C:/Users/killi/KF_Repo/PGA_Golf/Python_Scripts/Rugby_Union/2. Merge data/final_dataset.csv'

# Read in csv
final_dataset_df = pd.read_csv(final_dataset_loc)

# Drop columns
final_dataset_df = final_dataset_df.drop(columns=['date','key'])

# Rename columns
final_dataset_df = final_dataset_df.rename(columns={"date_2": "date"})

# Convert the 'date' column to datetime format
final_dataset_df['date'] = pd.to_datetime(final_dataset_df['date'],format='mixed')

# Format the 'date' column to 'yyyy-mm-dd' (remove the time)
final_dataset_df['date'] = final_dataset_df['date'].dt.strftime('%Y-%m-%d')

# Sort the 'date' column in ascending order
final_dataset_df = final_dataset_df.sort_values(by='date', ascending=True)

# Set Pandas option to display all rows (and avoid truncation)
pd.set_option('display.max_rows', None)

# Fill Nan with 0
final_dataset_df = final_dataset_df.fillna(0)

# Identify teams to remove based on counts

import numpy as np

home_teams = final_dataset_df['home_team'].unique()
away_teams = final_dataset_df['away_team'].unique()

# Concatenate the arrays and keep unique values
unique_teams = np.unique(np.concatenate((home_teams, away_teams)))

# get number of teams
n_teams = len(unique_teams)


# Initialize an empty list to store the results
team_counts = []

# Loop through each unique team and count occurrences in both columns
for team in unique_teams:
    home_count =  final_dataset_df['home_team'].isin([team]).sum()  # Count in 'home_team' column
    away_count =  final_dataset_df['away_team'].isin([team]).sum()  # Count in 'away_team' column
    total_count = home_count + away_count  # Total count for both columns
    team_counts.append({'team': team, 'count': total_count})

# Create a DataFrame from the results
team_counts_df = pd.DataFrame(team_counts)

# Print the resulting DataFrame
print(team_counts_df.sort_values(by='count', ascending=False))


# List of teams to remove - A teams and club teams
teams_to_remove = ['Scotland A','Reds','Munster', 'France A','England A','Australia A',
'Barbarians','New Zealand A','Maori All Blacks']

# Create copy of dataset
final_dataset_df_cleaned = final_dataset_df.copy()

# Remove rows where home team and away team column value is in the 'teams_to_remove' list
final_dataset_df_cleaned = final_dataset_df_cleaned[~final_dataset_df_cleaned['home_team'].isin(teams_to_remove)]
final_dataset_df_cleaned = final_dataset_df_cleaned[~final_dataset_df_cleaned['away_team'].isin(teams_to_remove)]



                    team  count
23           New Zealand     21
32          South Africa     20
0              Argentina     19
8                England     19
17                 Japan     17
30              Scotland     17
1              Australia     17
40                 Wales     17
10                  Fiji     16
15               Ireland     16
16                 Italy     15
11                France     15
35                 Tonga     12
7                  Chile     11
13               Georgia     10
39               Uruguay     10
29                 Samoa     10
36                   USA      9
28               Romania      9
26              Portugal      8
6                 Canada      7
34                 Spain      5
21               Namibia      5
5                 Brazil      5
14             Hong Kong      5
4                Belgium      3
41              Zimbabwe      2
25              Paraguay      2
19      Maori All Blacks      2
18                 Kenya      2
24      

In [6]:
# Now want to do checks on the data to see if there is any manual updates to make

# Combine Penalty goals and drop goals
final_dataset_df_cleaned['Home_pg_dg'] = (final_dataset_df_cleaned['Home_pg'] 
                                          + final_dataset_df_cleaned['Home_dg'])
final_dataset_df_cleaned['Away_pg_dg'] = (final_dataset_df_cleaned['Away_pg'] 
                                          + final_dataset_df_cleaned['Away_dg'])

# Need to update some of the conversion columns
# When there is a penalty try they don't count the conversions
final_dataset_df_cleaned['manual_score_home'] = (final_dataset_df_cleaned['Home_tries']*5 
                                + final_dataset_df_cleaned['Home_conv']*2
                                + final_dataset_df_cleaned['Home_pg_dg']*3)

final_dataset_df_cleaned['manual_score_away'] = (final_dataset_df_cleaned['Away_tries']*5 
                                + final_dataset_df_cleaned['Away_conv']*2
                                + final_dataset_df_cleaned['Away_pg_dg']*3)

# calculate differences
final_dataset_df_cleaned['Home_diff'] = final_dataset_df_cleaned['home_score'] - final_dataset_df_cleaned['manual_score_home'] 
final_dataset_df_cleaned['Away_diff'] = final_dataset_df_cleaned['away_score'] - final_dataset_df_cleaned['manual_score_away'] 


# Find examples where I have to update Home
# There is sometimes an issue with penalty tries so I update conversion manually
final_dataset_df_cleaned[final_dataset_df_cleaned['Home_diff'] != 0][['home_team',
                                                                      'away_team',
                                                                      'date',
    'home_score',
                                                                      'manual_score_home',
                                                                      'Home_diff',
                                                                      'Home_tries',
                                                                      'Home_conv',
                                                                      'Home_pg_dg'
                                                                     ]]

,home_team,away_team,date,home_score,manual_score_home,Home_diff,Home_tries,Home_conv,Home_pg_dg


In [3]:
# starting length
print(len(final_dataset_df_cleaned))

# Row indices to update
row_indices = [86,82,58,23,161,16,149,7,132,126,125,121,118] 

# Increase Home conversion by 1
final_dataset_df_cleaned.loc[row_indices, 'Home_conv'] += 1

# Row indices to update - subtrat 1 conversion
row_indices = [73] 

# Increase Home conversion by 1
final_dataset_df_cleaned.loc[row_indices, 'Home_conv'] -= 1

# Remove certain indexes where we don't have stats
remove_index = [160,153,139]
final_dataset_df_cleaned =final_dataset_df_cleaned.drop(remove_index)

# Final length
print(len(final_dataset_df_cleaned))



161
158


In [4]:
# Update away indexes

# Row indices to update
row_indices = [83,35,33,162,157,102,137,124] 

# Increase Away conversion by 1
final_dataset_df_cleaned.loc[row_indices, 'Away_conv'] += 1



In [5]:
# Now want to do checks on the data to see if there is any manual updates to make

# Combine Penalty goals and drop goals
final_dataset_df_cleaned['Home_pg_dg'] = (final_dataset_df_cleaned['Home_pg'] 
                                          + final_dataset_df_cleaned['Home_dg'])
final_dataset_df_cleaned['Away_pg_dg'] = (final_dataset_df_cleaned['Away_pg'] 
                                          + final_dataset_df_cleaned['Away_dg'])

# Need to update some of the conversion columns
# When there is a penalty try they don't count the conversions
final_dataset_df_cleaned['manual_score_home'] = (final_dataset_df_cleaned['Home_tries']*5 
                                + final_dataset_df_cleaned['Home_conv']*2
                                + final_dataset_df_cleaned['Home_pg_dg']*3)

final_dataset_df_cleaned['manual_score_away'] = (final_dataset_df_cleaned['Away_tries']*5 
                                + final_dataset_df_cleaned['Away_conv']*2
                                + final_dataset_df_cleaned['Away_pg_dg']*3)

# calculate differences
final_dataset_df_cleaned['Home_diff'] = final_dataset_df_cleaned['home_score'] - final_dataset_df_cleaned['manual_score_home'] 
final_dataset_df_cleaned['Away_diff'] = final_dataset_df_cleaned['away_score'] - final_dataset_df_cleaned['manual_score_away'] 

# Check Away diff
final_dataset_df_cleaned[final_dataset_df_cleaned['Away_diff'] != 0][['home_team',
                                                                      'away_team',
                                                                      'date',
                                                                        'away_score',
                                                                      'manual_score_away',
                                                                      'Away_diff',
                                                                      'Away_tries',
                                                                      'Away_conv',
                                                                      'Away_pg_dg'
                                                                     ]]

,home_team,away_team,date,away_score,manual_score_away,Away_diff,Away_tries,Away_conv,Away_pg_dg


In [7]:
# Reorder columns and remove some columns
final_dataset_df_cleaned = final_dataset_df_cleaned[[
    'date','home_team', 'away_team', 'home_score',
    'away_score', 'Home_tries',
       'Away_tries', 'Home_conv', 'Away_conv', 'Home_conv_att',
       'Away_conv_att', 'Home_pg', 'Away_pg', 'Home_pg_att', 'Away_pg_att',
       'Home_dg', 'Away_dg', 'Home_dg_att', 'Away_dg_att', 
       'Home_pg_dg', 'Away_pg_dg'
       ]]

In [8]:
len(final_dataset_df_cleaned)

158

In [9]:
# Now save this file so it can be used by model files
final_dataset_df_cleaned.to_csv('final_dataset_cleaned.csv',index=False)